<a href="https://colab.research.google.com/github/jssuriyakumar/Machine-Learning---AI/blob/main/Creating_custom_methods_using_OOPS_in_SKLEARN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OOPS refresher

In [1]:
class MeanImputer:
  pass

In [2]:
class MeanImputer:
  
  def __init__(self, variables): # all classes take an init class that specifies the properties that the class takes whenever it is initialzied. the first param is always self (calling it self)
    self.variables = variables


In [3]:
myImputer = MeanImputer(['age','sex'])

In [5]:
myImputer.variables # accessing the class attributes    

['age', 'sex']

In [6]:
class MeanImputer:

  def __init__(self, variables):
    self.variables = varaiables
  
  def fit(self,X, y=None): # in our example fit method learns the parameters (learns the function and computes the function value)
    self.imputerDict = X[self.variables].mean().to_dict()
    return self # means not returning anything

  def transform(self, X): #u in our example transform method applies the learned parameters to the dataset
    for i in self.variables:
      X[i] = X[i].fillna(self.imputerDict[i])
      return X

Inheritance in SCIKIT LEARN API

In [10]:
# process of one class taking methods and attributes of other class

# PARENT CLASS
class TransformerMixin:
  def fit_transform(self, X, y = None):
    X = self.fit(X,y).transform(X)
    return X

# CHILD CLASS 
class MeanImputer(TransformerMixin): # child class inheriting the methods of parent class

  def __init__(self, variables):
    self.variables = variables
  
  def fit(self,X, y=None): # in our example fit method learns the parameters (learns the function and computes the function value)
    self.imputerDict = X[self.variables].mean().to_dict()
    return self # means not returning anything

  def transform(self, X): #u in our example transform method applies the learned parameters to the dataset
    for i in self.variables:
      X[i] = X[i].fillna(self.imputerDict[i])
      return X

In [ ]:
# Now since the parent class is inherited the we can call the methods of parent class directly from the child class
myImputer = MeanImputer(['age','sex']).fit_transform(X,y)

https://scikit-learn.org/stable/modules/classes.html 

Scikit learn has classes to make or extend this classes with user specific or problem specific classes

For most of the preprocessing we have to sue BaseEstimator and Transformermixin

Most of the preprocessing activities are problem specific no ready made class is available to tackle this we have to create our own classes

In [13]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin

In [14]:
# LET US CREATE A USER DEFINED CLASS TO CALCULATE TEMPORAL DIFFERENCE (DIFFERENCE BETWEEN TWO TIMEFRAMES TO REDUCE DIMENSIONALITY)

class TemporalDifference(BaseEstimator,TransformerMixin): # named a class as TemporoalDifference (child class)

  def __init__(self,variables, reference):
    # type check
    if not isinstance(variables, list):
      raise ValueError(f'variables expected as a list got {type(variables)}')
    if not isinstance(reference, str):
      raise ValueError(f'reference expected as a string got {type(variables)}')

    self.variables = variables
    self.reference = reference 

  def fit(self): # though we are not calculating any value in prior which will be applied in transform, it is neccessary to keep fit method as per sklearn requirement

    return self
  def transform(self, X, y = None):

    X = X.copy()
    if self.variables not in X.columns:
      raise ValueError('One or more columns not found in the dataframe')
    for i in self.variables:
      X[i] = X[i] - X[self.reference]
    return X

In [ ]:
# LET US CREATE A USER DEFINED CLASS TO REPLACE THE STRING BASED CATEGORICAL VARIABLES TO USER-DEFINED FORMAT WHICH WILL BE USED ALMOST EVERYWHERE IN THE DATA SCIENCE PROBLEM

class Mapper(BaseEstimator,TransformerMixin):

  def __init__(self, variables, mappings):
    # type check
    if not isinstance(variables, list):
      raise ValueError(f'variables expected as a list got {type(variables)}')
    
    self.variables = variables
    self.mappings = mappings
  
  def fit(self): # though we are not calculating any value in prior which will be applied in transform, it is neccessary to keep fit method as per sklearn requirement
    return self

  def transform(self, X, y = None):
    X = X.copy()
    if self.variables not in X.columns:
      raise ValueError('One or more columns not found in the dataframe')
    for i in self.variables:
      X[i] = X[i].map(self.mappings)
    return X

In [ ]:
# LET US CREATE TRANSFORMERS THAT CAN LEARN AND STORED DATA

class MeanImputer(BaseEstimator, TransformerMixin):

  def __init__(self, variables):
    if not isinstance(variables, list):
      raise ValueError(f'variables expected as a list got {type(variables)}')
    self.variables = variables
  
  def fit(self,X, y=None): # in our example fit method learns the parameters (learns the function and computes the function value)
    self.imputerDict = X[self.variables].mean().to_dict()
    return self # means not returning anything

  def transform(self, X): #u in our example transform method applies the learned parameters to the dataset
    for i in self.variables:
      X[i] = X[i].fillna(self.imputerDict[i],inplace=True)
      return X

In [15]:
# LET US CREATE A USER DEFINED CLASS TO REPLACE THE STRING BASED CATEGORICAL VARIABLES TO USER-DEFINED FORMAT WHICH WILL BE USED ALMOST EVERYWHERE IN THE DATA SCIENCE PROBLEM

class RareCategoricalEncoder(BaseEstimator, TransformerMixin):
  
  def __init__(self,variables,limit = 0.03):
    
    if not isinstance(variables, list):
      raise ValueError(f'variables expected as a list got {type(variables)}')
    self.variables = variables
    self.limit = limit

  def fit(self, X, y=None):

    self.encoder_dict = {}
    for i in self.variables:
      variable_list = pd.Series(X[i].value_counts(normalize = True))
      self.encoder_dict[i] = list(variable_list[variable_list>self.limit].index)
    return self

  def transform(self, X):
    X = X.copy()
    for i in self.variables:
      X[i] = np.where(X[i].isin(self.encoder_dict[i],X[i],"Rare"))
    return X
    

In [16]:
# LET US CREATE A USER DEFINED CLASS TO PERFORM CATEGORICAL ENCODING
class CategoricalEncoder(BaseEstimator, TransformerMixin):
  
  def __init__(self,variables):
    if not isinstance(variables, list):
      raise ValueError(f'variables expected as a list got {type(variables)}')
    self.variables = variables
  
  def fit(self, X, y):
    temp = pd.concat([X,y],axis = 1)
    temp.columns = list(X.columns) + ['target']
    
    self.encoder_dict = {}
    for i in self.variables:
      t = temp.groupby([i])['target'].mean().sort_values(ascending=True).index
      self.encoder_dict[i] = {k:j for j,k in enumerate(t,0)}
      return self
    
  def transform(self, X):

    X = X.copy()
    for i in self.variables:
      X[i] = X[i].map(self.encoder_dict[i])
    return X
    